<a href="https://colab.research.google.com/github/ajit34/MP34/blob/main/decision-opt-course/assignment/start_here.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/wandb/edu/blob/main/decision-opt-course/assignment/start_here.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{decisionopt-assignment} -->

# Data

In [1]:
# Uncomment if you're on Google Colab
# !mkdir data
# !wget -P data https://raw.githubusercontent.com/wandb/edu/main/decision-opt-course/assignment/data/train.csv
# !pip install wandb -qqq
# !pip install scikit-learn==1.1.3 -qqq

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/train.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'

Note: TWF, HDF, PWF, OSF and RNF are different types of failures (eg tool wear, heat dissipation, power wear, overstrain and random failures respectively). You will decide whether to repair a machine before the failure, and thus before you would know `Machine failure` or any of the failure type information

# Your Mission
<!--- @wandbcode{decisionopt-assignment} -->

Your mission, should you choose to accept it, is to write a python function that will determine when a large manufacturer repairs their machines and when they let them run unrepaired 🙏🏻

You will use data from Kaggle's [Classication of Machine Failures](https://www.kaggle.com/competitions/playground-series-s3e17) competition. The original data doesn't include economic information like the cost of repairing a machine. This information is described in this assignment and you will code it in your simulation (the original data was also synthetically generated, so it was generated with a similar process).

You will

1. Write a function that calculates maintenance and repair costs when you use any supplied decision function for determining when to do machine maintenance
2. Create 3 decision functions and compare their costs using the function you just wrote in step 1
3. Write and share a Weights & Biases report so your colleagues can see the implications of using your preferred decision function. In your report, describe (in just a few sentences):
  - How you might deal with potential covariate shift if training data was collected in winter when air temperatures are lower, so you expect future air temperatures to be warmer
  - How you might test your decision function in reality to make sure it works before applying it at scale

In [ ]:
# Add any EDA you want to do here. You do not need to include it in the final report

# Modeling

You can use this model or change it to another model you like

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector

predictive_features = ['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
# We split data between train and test by indexing into data.
# This splits by time period if data came in chronological order.
# It's also useful to keep this as a DataFrame so your decision function can access features by name
train = data[:100_000]
test = data[100_000:]

preprocessor = ColumnTransformer(
    transformers=[("one_hot", OneHotEncoder(), selector(dtype_include="object"))],
    remainder="passthrough",  # Leave numerical variables unchanged
)
modeling_pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", GradientBoostingClassifier())])
modeling_pipeline.fit(train[predictive_features], train['Machine failure'])

In [ ]:
import wandb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, plot_roc_curve

project_name = 'Decision Optimization Assignment'
# start a new run
run = wandb.init(project=project_name)

# predict labels for test set
test_predictions = modeling_pipeline.predict(test[predictive_features])
test_probabilities = modeling_pipeline.predict_proba(test[predictive_features])[:, 1]

# calculate metrics
conf_mat = confusion_matrix(test['Machine failure'], test_predictions)
accuracy = accuracy_score(test['Machine failure'], test_predictions)
precision = precision_score(test['Machine failure'], test_predictions)
recall = recall_score(test['Machine failure'], test_predictions)
f1 = f1_score(test['Machine failure'], test_predictions)
roc_auc = roc_auc_score(test['Machine failure'], test_probabilities)

# calculate feature importance for tree-based model
feature_importance = modeling_pipeline.named_steps['classifier'].feature_importances_

# log metrics to wandb
wandb.log({"confusion_matrix": wandb.plots.HeatMap(
    np.unique(test['Machine failure']),
    np.unique(test['Machine failure']),
    conf_mat,
    show_text=True
)})
wandb.log({"accuracy": accuracy})
wandb.log({"precision": precision})
wandb.log({"recall": recall})
wandb.log({"f1": f1})
wandb.log({"roc_auc": roc_auc})
wandb.log({"feature_importance": wandb.Histogram(feature_importance)})

# plot roc curve
ax = plot_roc_curve(modeling_pipeline, test[predictive_features], test['Machine failure'])
ax.figure_.savefig("roc_curve.png")
wandb.log({"roc_curve": wandb.Image("roc_curve.png")})

# end run
run.finish()


# Simulation

Your model will predict the likelihood of failure for each row in the data. You will then apply a decision function that considers `Machine Type`` and the probability of failure to decide whether to do preventive maintenance for that machine at that time.

If choose to do preventive maintenance, you pay the cost shown below for `Cost of preventive maintenance` for that machine type. If you don't do maintenance and the observed data in validation is that it failed, then you pay `Cost of failure` for that machine type.

| Machine Type | Cost of preventive maintenance | Cost of failure if you don't do preventive maintenance |
|--------------|--------------------------------|--------------------------------------------------------|
| L            | 1000                           | 30000                                                  |
| M            | 5000                           | 250000                                                 |
| H            | 10000                          | 800000                                                 |

First, write a function that determines the costs given some data, a machine learning model to predict failure rates, and a decision function:

In [ ]:
def cost_calculator(data, model, decision_function):
    """
    Args:
        data (pandas.DataFrame): The dataset on which the cost is calculated.
        model (sklearn-compatible model. Compatible with type sklearn.base.BaseEstimator): The predictive model.
        decision_function (function): The decision function of the model.

    Returns:
        int: The cost of using this decision function with the predictive model on this dataset.
    """
    # YOUR CODE HERE

Here is an example decision function.

In [ ]:
def first_decision_function(data, model):
    """
    Args:
        data (pandas.DataFrame): Data to make a decision on
        model (sklearn model): A trained sklearn classifier

    Returns:
        repair_decision (bool): A decision to repair or not to repair
    """
    fixed_decision_threshold = 0.01
    probability_of_failure = model.predict_proba(data[predictive_features])[:, 1]
    repair_decision = probability_of_failure > fixed_decision_threshold
    return repair_decision

first_costs = cost_calculator(test, modeling_pipeline, first_decision_function)

# Log this with wandb. Then add and test two more decision functions.
# You should be able to find a better decision function than the first one.
# Do you see how to do better than just changing `fixed_decision_threshold` to a different number?

Once you have created and compared at least three decision functions, you can create and submit your report comparing them. Remember to describe (in a sentence or two) your plan for dealing with the covariate shift and testing issues described at the top of this assignment.

Good Luck!